## 0. 核心概念

- 在使用Matlab对数据进行预处理时，遇到了内存不足的问题，因为数据量太大，在处理完成以前内存已经爆满。如果使用Matlab的.m文件对文件进行存储的话，则需要将数据分割成多个文件，对后续的处理造成了不便。HDF5文件则是一种灵活的文件存储格式，有一个最大的好处就是在Matlab的处理过程中可以对它进行扩展写入，也就是说不是所有数据处理完以后一次写入，而是边处理边写入，极大的降低了对系统内存的要求。

- HDF5文件类似与一个文件系统，使用这个文件本身就可以对数据集（dataset）进行管理。例如下图所示，HDF5文件中的数据集皆存储根目录/，在根目录下存在多个group，这样一些group类似与文件系统的文件夹，在它们可以存储别的group，也可以存储数据集。


![title](picture/001.png)


对于每一个dataset 而言，除了数据本身之外，这个数据集还会有很多的属性 attribute.在hdf5中，还同时支持存储数据集对应的属性信息，所有的属性信息的集合就叫做metadata;

![title](picture/002.png)

# 1 使用matlab处理相关hdf5

## 1.1 创建文件

使用Matlab创建HDF5文件的函数是h5create，使用如下：

> h5create(filename,datasetname,[30, 30 , 3, inf],'Datatype','single','ChunkSize',[30,30,3,1000])

- filename为h5文件的文件名（不知道什么问题，在我的电脑上使用时，这个函数无法指定路径）。
- datasetname则为数据集的名字，数据集名称必须以/开头，比如/G。
 [30,30,3,inf]位数据集的大小，比如我的数据集为30x30大小的彩色图像，并且我希望数量能够扩展，那么就可以指定最后以为度为inf，以表示数量不限。
- Datatype为数据类型
- ChunkSize为数据存储的最小分块，为了让数据能够具有扩展性，所以为新来的数据分配一定的空间大小，对于一个非常大的数据，这个值设置大一点比较好，这样分块就会少一点。比如我的数据集中，30x30大小的彩色图像大概有10万个左右，那么1000个存储在一起较为合适，则chunksize设置为：[30,30,3,1000]。


## 1.2 使用Matlab写入HDF5

在创建了hdf5文件和数据集以后，则可以对数据集进行写操作以扩展里面的数据。使用Matlab写入HDF5文件的函数是h5write，使用如下：

> h5write(fileName,datasetName,data,start,count);

- fileName: hdf5文件名
- datasetName:数据集名称，比如/G
- data:需要写入的数据，数据的维度应该与创建时一致，比如，设置的数据集大小为[30,30,3,inf]，那么这里的data的前三个维度就应该是[30, 30, 3]，而最后一个维度则是自由的
- start:数据存储的起点，如果是第一次存，则应该为[1, 1, 1, 1]（注意数据维度的一致性），如果这次存了10000个样本，也就是[30,30,3,10000]，那么第二次存储的时候起点就应该为[1,1,1,10001]
- count存储数据的个数，同样要根据维度来（其实就是数据的维度），这里为[30,30,3,10000]


## 1.3 使用Matlab读取HDF5中的数据集

Matlab中可以使用h5info函数来读取HDF5文件的信息：

> fileInfo = h5info(fileName);

然后通过解析fileInfo结构，则可以得到HDF5文件中的数据集名称、数据集大小等等必要信息。

## 1.4 使用Matlab读取HDF5中的数据集

Matlab中可以使用h5read函数来读取HDF5文件：

> data = h5read(filename,datasetname,start,count)

- filename：HDF5文件文件名
- datasetname：数据集名称
- start：从数据集中取数据的其实位置
- count：取的数据数量
- 还是以上面的30x30的彩色图像为例，如果每次需要取1000个，那么第一次取时，start应该设置为[1, 1, 1, 1] ，count设置为：[30, 30 ,3 1000]。第二次取值时，start则应该设置为[1, 1, 1, 1001]，count则设置为：[30, 30, 3, 1000]。



# 2 使用python处理hdf5文件

- 一个HDF5文件是一种存放两类对象的容器：dataset和group. Dataset是类似于数组的数据集，而group是类似文件夹一样的容器，存放dataset和其他group。在使用h5py的时候需要牢记一句话：groups类比词典，dataset类比Numpy中的数组。 
- HDF5的dataset虽然与Numpy的数组在接口上很相近，但是支持更多对外透明的存储特征，如数据压缩，误差检测，分块传输。

## 2.1 创建和保存文件

### 2.1.1 创建一个HDF5文件

In [1]:
import h5py  # 首先我们导入h5dy的包
import numpy as np
f = h5py.File("mytestfile.hdf5", "w")  # 首先我们以'w'模式写入一个文件，然后保存为mytestfile.hdf5

#### 2.1.1.1 创建一个dataset

我们可以借助文件对象的一系列方法添加数据。其中create_dataset用户创建给定形状和数据类型的空dataset.

In [2]:
dset = f.create_dataset("mydataset", (100,), dtype='i')

> create_dataset(name, shape=None, dtype=None, data=None, **kwds)

Create a new dataset. Options are explained in Creating datasets.

**Parameters:**	
- name – Name of dataset to create. May be an absolute or relative path. Provide None to create an anonymous dataset, to be linked into the file later.
- shape – Shape of new dataset (Tuple).
- dtype – Data type for new dataset
- data – Initialize dataset to this (NumPy array).
- chunks – Chunk shape, or True to enable auto-chunking.
- maxshape – Dataset will be resizable up to this shape (Tuple). Automatically enables chunking. Use None for the axes you want to be unlimited.
- compression – Compression strategy. See Filter pipeline.
- compression_opts – Parameters for compression filter.
- scaleoffset – See Scale-Offset filter.
- shuffle – Enable shuffle filter (T/F). See Shuffle filter.
- fletcher32 – Enable Fletcher32 checksum (T/F). See Fletcher32 filter.
- fillvalue – This value will be used when reading uninitialized parts of the dataset.
- track_times – Enable dataset creation timestamps (T/F).

我们可以使用现有Numpy数组来初始化一个dataset

In [6]:
arr = np.arange(100)
dset = f.create_dataset("init", data=arr)

#### 2.1.1.2 分块存储策略

- 在缺省设置下，HDF5数据集在内存中是连续布局的，也就是按照传统的C序。Dataset也可以在HDF5的分块存储布局下创建。也就是dataset被分为大小相同的若干块随意地分布在磁盘上，并使用B树建立索引。 

为了进行分块存储，将关键字设为一个元组来指示块的形状。

In [ ]:
dset = f.create_dataset("chunked",(1000,1000),chunks=(100,100))

也可以自动分块，不必指定快的形状

In [7]:
dset  = f.create_dataset("autochunk",(1000,1000), chunks=True)

### 2.2 读取HDF5文件的内容

In [17]:
import h5py
f = h5py.File('mytestfile.hdf5','r')  # 这里是'r'表示我们读取hdf5里面的内容

请记住htpy.File类似python的dict对象，因此我们可以查看所有的键值：


In [18]:
f.keys()

KeysView(<HDF5 file "mytestfile.hdf5" (mode r+)>)

基于以上观测，文件中有名字为mydataset这样一个数据集。然后我们可以用类似词典的方法读取对应的dataset对象

In [24]:
dset = f['mydataset']  # 先读取
dset.shape  # 我们可以输出他的维度
dest.dtype # 也可以得到的数据类型

dtype('int64')

#### 2.3 HDF5的分层结构

“HDF”代表"Hiearchical Data Format(分层数据格式)".hdf5文件中group类似于文件夹，我们创建的文件对象本身就是一个group，称为root group.

In [8]:
f.name  # 因为这里我们还没有创建name，所以暂时为空

'/'

创建subgroup是使用create_group的方法实现的，但是我们首先需要用读写模式打开文件：


In [25]:
f = h5py.File('mytestfile.hdf5','r+')
grp = f.create_group('subgroup')

我们在group中跌倒就可以得到group内所有的直接附属的成员，包含dataset和subgroup

In [28]:
for name in f:
    print(name)

autochunk
init
mydataset
subgroup


### 2.4 属性

HDF5的一个很棒的特点就是你可以在数据旁边存储元数据,所有的group和dataset都支持叫做属性的数据格式,属性通过attrs成员访问，类似与python词典格式。

In [29]:
dset.attrs['temperature'] = 99.5

In [31]:
dset.attrs['temperature']

99.5

## 3. 高级特征

### 3.1 滤波器组

* HDF5的滤波器组可以对分块数组进行变换，最常用的变换是高保真压缩,使用一个特定的压缩滤波器创建dataset之后，读写都可以向平常一样，不必添加额外的步骤。
* 用管检测compression来指定压缩滤波器,而滤波器可以利用关键词compression_opt指定：


In [34]:
dset = f.create_dataset('zipped', (100,100),compression='gzip')

### 3.2 HDF5文件的限制

- HDF5文件本身大小没有限制，但是HDF5的一个dataset最高允许32个维，每个维度最多可有2^64个值，每个值大小理论上可以任意大  
- 目前一个chunk允许的最大容量为2^32-1 byte (4GB). 大小固定的dataset的块的大小不能超过dataset的大小。


## 4 参考文献

1. [h5py quick start guide](http://docs.h5py.org/en/latest/quick.html)
2. [python开源库——h5py快速指南](https://blog.csdn.net/yudf2010/article/details/50353292)